In [1]:
import tensorflow as tf
import model

X = ["this is some text", "this is a bit more"]
y = tf.constant([[1, 0, 1, 0], [1, 1, 0, 0]], dtype=tf.float32)

2021-11-07 21:30:28.191346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-07 21:30:28.237013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-07 21:30:28.237165: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-07 21:30:28.238401: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [2]:
OUTPUT_SIZE = y.shape[-1]

FAKE_BATCH_SIZE = 2
FAKE_INPUT_SIZE = 100

In [3]:
bert = model.BertEncoder(
    tfhub_handle_preprocess="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    tfhub_handle_encoder="https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1",
)

bert_sample = bert(X)
bert_output_size = bert_sample.shape[1]

fake_sample = tf.random.normal(
    (FAKE_BATCH_SIZE, FAKE_INPUT_SIZE),
    mean=0.0, stddev=1.0,
    dtype=tf.float32
)

2021-11-07 21:30:29.801880: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [4]:
generator = model.GeneratorModel(hidden_size=bert_output_size)
discriminator = model.DiscriminatorModel(
    hidden_size=bert_output_size,
    output_size=OUTPUT_SIZE
)

In [5]:
generator_logits = generator(fake_sample)

In [13]:
real = discriminator(bert_sample)
fake = discriminator(generator_logits)

In [14]:
is_fake_probabilities_real = real["probabilities"][:, 0]
is_fake_probabilities_fake = fake["probabilities"][:, 0]
class_probabilities_real = real["probabilities"][:, 1:]

In [15]:
epsilon = 0.001

loss_supervised = -tf.math.reduce_mean(y[:, 1:] * tf.math.log(class_probabilities_real + epsilon))
loss_unsupervised_real = -tf.math.reduce_mean(1 - tf.math.log(is_fake_probabilities_real + epsilon))
loss_unsupervised_fake = -tf.math.reduce_mean(tf.math.log(is_fake_probabilities_fake + epsilon))

discriminator_loss = loss_supervised + loss_unsupervised_real + loss_unsupervised_fake

In [20]:
generator_loss = -tf.math.reduce_mean(1 - tf.math.log(is_fake_probabilities_fake + epsilon))

generator_features = tf.math.reduce_mean(fake["hidden-layer"], axis=0)
real_features = tf.math.reduce_mean(real["hidden-layer"], axis=0)
feature_loss = tf.math.reduce_mean(tf.math.square(generator_features - real_features))

generator_loss = generator_loss + feature_loss

In [29]:
print(f"The generator loss {generator_loss.numpy()}")
print(f"The generator loss {discriminator_loss.numpy()}")

The generator loss -2.544656991958618
The generator loss 1.0323132276535034
